In [4]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 KB 3.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import re
import nltk
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import concurrent

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

In [11]:
df = pd.read_excel('/content/drive/MyDrive/Research papers/Thesis /Data/processed data/twitter/data_new_last.xlsx')

In [13]:
df.columns

Index(['Unnamed: 0', 'ticker', 'tweet', 'date', 'likes'], dtype='object')

In [14]:
### cleaning
def clean_text(i, text):
    #basic
    text = str(text)
    text = text.lower()
    text = re.sub(' +', ' ', text)
    text = re.sub('\n', '', text)
    text = re.sub(r"(http?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)","", text)
    text = re.sub(r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)","", text)
    text = re.sub('[^a-zA-Z]+\s*', ' ', text)


    #lemmatization
    
    
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])

    #stopwords removal 

    text = [word for word in text.split(' ') if word not in stop_words]
    text = ' '.join(text)

    df['tweet'][i] = text

    if i % 1000 == 0:
      print(f"[INFO]: {i} is done!")
    

In [15]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    for i, text in enumerate(df['tweet'].values):
        executor.submit(clean_text, i, text)

    


<ipython-input-14-926c0e660b69>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i] = text


In [16]:
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']

In [17]:
df['sentiment'] = df['tweet'].apply(sentiment_scores)

In [18]:
sent_df = df.groupby('ticker')['sentiment'].mean()

In [19]:
sent_df = pd.DataFrame(sent_df).reset_index()

In [20]:
names = pd.read_csv('/content/drive/MyDrive/Research papers/Thesis /Data/processed data/twitter/ticker2name.csv')

In [21]:
for i in range(len(names)):
  if pd.isnull(names['sentiment'][i]):
    ticket = names['ticker'][i]
    try:
      names['sentiment'][i] = sent_df[sent_df['ticker'] == ticket]['sentiment'].values[0]

    except:
       continue

<ipython-input-21-1bdd08f0b9be>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  names['sentiment'][i] = sent_df[sent_df['ticker'] == ticket]['sentiment'].values[0]


In [23]:
names.to_csv('/content/drive/MyDrive/Research papers/Thesis /Data/processed data/twitter/sentiment_scores.csv', index = None)